In [1]:
import brightway2 as bw
import os               
import numpy as np       
import pandas as pd     
import time
from brightway2 import *
from bw_recipe_2016 import add_recipe_2016
import bw2analyzer as bwa
import stats_arrays
import matplotlib.pyplot as plt

In [2]:
bw.projects.set_current("EIDB_only")

In [3]:
bw.projects.dir

'C:\\Users\\ruqun\\AppData\\Local\\pylca\\Brightway3\\EIDB_only.52543ebe294165861341ee605773a288'

In [4]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.6 cutoff
	ecoinvent 3.7 cutoff

In [5]:
eidb = bw.Database('ecoinvent 3.7 cutoff')

In [6]:
my_bio = bw.Database('biosphere3')

## processing eidb, finding unique "ref. prod"

In [7]:
len(eidb)  

19128

In [8]:
alleidb = [act for act in eidb if act['reference product'] != None and "ordinary transforming activity" in act['activity type'] ]
len(alleidb)   #same 13017 in raw excel by incl. only "ordinary transforming activity"

13017

In [9]:
allref_list = []
for prod in alleidb: 
    allref_list.append(prod.as_dict()['reference product'])

In [10]:
#create a new DF for storing
#there are 78 product with "ISIC number" = "not available now", for scrap/waste prod. they're not included as they have 0 impacts

allname_list = []
allref_list = []
allprod_cat_list = []
for prod in alleidb: 
    for i in range(0,len(prod.as_dict()['classifications'])):
        if prod.as_dict()['classifications'][i][0] == "ISIC rev.4 ecoinvent":
            allname_list.append(prod.as_dict()['name'])
            allref_list.append(prod.as_dict()['reference product'])
            allprod_cat_list.append(prod.as_dict()['classifications'][i][1])

In [11]:
print(len(allprod_cat_list)) 
print(len(set(allref_list))) 
print(len(set(allprod_cat_list))) 

12939
3167
208


In [12]:
#for each ref_prod, if more then one transforming activity exist, then only one activity is displaced as an example
allprodnotunique = {'ref_prod':allref_list, 'activity': allname_list, 'ISIC_cat':allprod_cat_list}

In [13]:
df_allprodnotunique = pd.DataFrame(allprodnotunique)
df_allprodnotunique

,ref_prod,activity,ISIC_cat
0,"inorganic phosphorus fertiliser, as P2O5",nutrient supply from single superphosphate,0161:Support activities for crop production
1,"organic nitrogen fertiliser, as N",nutrient supply from fermentation filtrate,0161:Support activities for crop production
2,"inverter, 2.5kW","inverter production, 2.5kW",2790:Manufacture of other electrical equipment
3,"water pump operation, diesel","water pump operation, diesel",0161:Support activities for crop production
4,"sulfidic tailings, generic","treatment of sulfidic tailings, generic, taili...",3822:Treatment and disposal of hazardous waste
...,...,...,...
12934,waste glass pane in burnable frame,treatment of waste glass pane in burnable fram...,3821:Treatment and disposal of non-hazardous w...
12935,explosives factory,explosives factory construction,4290:Construction of other civil engineering p...
12936,"transport, freight, lorry with refrigeration m...","transport, freight, small lorry with refrigera...",4923:Freight transport by road
12937,corrugated board box,corrugated board box production,1702:Manufacture of corrugated paper and paper...


must first sort by "ref_prod" then by "cat", otherwise, freq_index mess up when counting freq. for each unique product

In [14]:
df_allprod_sorted = df_allprodnotunique.sort_values(['ref_prod', 'ISIC_cat']).reset_index(drop=True)
df_allprod_sorted

,ref_prod,activity,ISIC_cat
0,"1,1-difluoroethane, HFC-152a","1,1-difluoroethane production, HFC-152a",2011:Manufacture of basic chemicals
1,"1,1-difluoroethane, HFC-152a","1,1-difluoroethane production, HFC-152a",2011:Manufacture of basic chemicals
2,"1,1-dimethylcyclopentane",molecular sieve separation of naphtha,1920:Manufacture of refined petroleum products
3,"1,1-dimethylcyclopentane",molecular sieve separation of naphtha,1920:Manufacture of refined petroleum products
4,1-butanol,"synthetic fuel production, from coal, high tem...",1920:Manufacture of refined petroleum products
...,...,...,...
12934,"zircon, 50% zirconium",heavy mineral sand quarry operation and titani...,0729:Mining of other non-ferrous metal ores
12935,"zircon, 50% zirconium",heavy mineral sand quarry operation,0729:Mining of other non-ferrous metal ores
12936,zirconium oxide,zirconium oxide production,2011:Manufacture of basic chemicals
12937,zirconium oxide,zirconium oxide production,2011:Manufacture of basic chemicals


In [15]:
pd.DataFrame(df_allprod_sorted).to_excel('eidb_cutoff_all_12727refprod_wo_recycledcontent.xlsx')

In [16]:
uniq_3167_refprod = df_allprod_sorted.ref_prod.unique()
print(len(uniq_3167_refprod))
print(len(df_allprodnotunique.activity.unique()))  
#total 4162 unique activities > unique ref. product (N = 3167) as one ref.prod can be produced from diff. transforming activities
print(len(df_allprodnotunique.ISIC_cat.unique()))
#df_allprodnotunique.ISIC_cat.unique()

3167
4162
208


### count Freq of each unique product

In [17]:
uniq_refprod_index = []
for uniqprod in uniq_3167_refprod:
    uniq_refprod_index.append(list(df_allprod_sorted.ref_prod).count(uniqprod))

In [18]:
len(uniq_refprod_index)

3167

#uniq_refprod_index:  highest freq. for electricity, high voltage = 1911

In [19]:
uniq_refprod_2col = {'ref_prod':uniq_3167_refprod, 'count_prod': uniq_refprod_index} 

In [20]:
df_uniq_refprod_2col = pd.DataFrame(uniq_refprod_2col)
df_uniq_refprod_2col.sort_values('count_prod')

,ref_prod,count_prod
1583,"mounting, surface mount technology, Pb-contain...",1
1445,"maintenance, tanker, for petroleum",1
1444,"maintenance, tanker, for liquefied natural gas",1
1443,"maintenance, tanker for liquid goods other tha...",1
1439,"maintenance, refrigeration machine",1
...,...,...
1293,"land tenure, arable land, measured as carbon n...",175
751,"electricity, medium voltage",224
1129,"heat, district or industrial, other than natur...",371
749,"electricity, low voltage",487


### adding a new col to df_allprod_sorted, indicating freq. of each unique ref. prod

In [21]:
import itertools
from itertools import repeat, chain

In [22]:
n_value = df_uniq_refprod_2col.count_prod
freq_index = list(itertools.chain(*(itertools.repeat(elem, n) for elem, n in zip(n_value, n_value))))

In [23]:
print(len(freq_index))
len(freq_index) == len(df_allprod_sorted)

12939


True

In [24]:
df_allprod_sorted['prod_freq'] = freq_index
df_allprod_sorted

,ref_prod,activity,ISIC_cat,prod_freq
0,"1,1-difluoroethane, HFC-152a","1,1-difluoroethane production, HFC-152a",2011:Manufacture of basic chemicals,2
1,"1,1-difluoroethane, HFC-152a","1,1-difluoroethane production, HFC-152a",2011:Manufacture of basic chemicals,2
2,"1,1-dimethylcyclopentane",molecular sieve separation of naphtha,1920:Manufacture of refined petroleum products,2
3,"1,1-dimethylcyclopentane",molecular sieve separation of naphtha,1920:Manufacture of refined petroleum products,2
4,1-butanol,"synthetic fuel production, from coal, high tem...",1920:Manufacture of refined petroleum products,3
...,...,...,...,...
12934,"zircon, 50% zirconium",heavy mineral sand quarry operation and titani...,0729:Mining of other non-ferrous metal ores,4
12935,"zircon, 50% zirconium",heavy mineral sand quarry operation,0729:Mining of other non-ferrous metal ores,4
12936,zirconium oxide,zirconium oxide production,2011:Manufacture of basic chemicals,2
12937,zirconium oxide,zirconium oxide production,2011:Manufacture of basic chemicals,2


### extract only unique ref. product from df_allprod_sorted

#need to deal with the situation when multipe ISIC categories involved for the same RP, in such case, new value "various_ISIC" assigned to ISIC_cat.  

This happens for example: under "1920:Manufacture of refined petroleum products" which produced various chemicals (e.g., 1-butanol, benzene, chemical, organic), these chemicals can also be produced by conventional methods under "2011:Manufacture of basic chemicals"

In [25]:
df_3167uniprod = df_allprod_sorted.drop_duplicates(subset = ["ref_prod"]).reset_index(drop = True)
df_3167uniprod 

,ref_prod,activity,ISIC_cat,prod_freq
0,"1,1-difluoroethane, HFC-152a","1,1-difluoroethane production, HFC-152a",2011:Manufacture of basic chemicals,2
1,"1,1-dimethylcyclopentane",molecular sieve separation of naphtha,1920:Manufacture of refined petroleum products,2
2,1-butanol,"synthetic fuel production, from coal, high tem...",1920:Manufacture of refined petroleum products,3
3,1-pentanol,hydroformylation of butene,2011:Manufacture of basic chemicals,2
4,1-propanol,"synthetic fuel production, from coal, high tem...",1920:Manufacture of refined petroleum products,3
...,...,...,...,...
3162,zinc slag,"treatment of zinc slag, residual material land...",3822:Treatment and disposal of hazardous waste,1
3163,zinc sulfide,cobalt production,0729:Mining of other non-ferrous metal ores,3
3164,"zircon, 50% zirconium",heavy mineral sand quarry operation,0729:Mining of other non-ferrous metal ores,4
3165,zirconium oxide,zirconium oxide production,2011:Manufacture of basic chemicals,2


In [26]:
df_3457uniprodWHdiffISIC = df_allprod_sorted.drop_duplicates(subset = ["ref_prod", "ISIC_cat"])
df_3457uniprodWHdiffISIC

,ref_prod,activity,ISIC_cat,prod_freq
0,"1,1-difluoroethane, HFC-152a","1,1-difluoroethane production, HFC-152a",2011:Manufacture of basic chemicals,2
2,"1,1-dimethylcyclopentane",molecular sieve separation of naphtha,1920:Manufacture of refined petroleum products,2
4,1-butanol,"synthetic fuel production, from coal, high tem...",1920:Manufacture of refined petroleum products,3
5,1-butanol,hydroformylation of propylene,2011:Manufacture of basic chemicals,3
7,1-pentanol,hydroformylation of butene,2011:Manufacture of basic chemicals,2
...,...,...,...,...
12929,zinc sulfide,cobalt production,0729:Mining of other non-ferrous metal ores,3
12930,zinc sulfide,zinc sulfide production,2011:Manufacture of basic chemicals,3
12932,"zircon, 50% zirconium",heavy mineral sand quarry operation,0729:Mining of other non-ferrous metal ores,4
12936,zirconium oxide,zirconium oxide production,2011:Manufacture of basic chemicals,2


In [27]:
#df_3457uniprodWHdiffISIC.duplicated(subset=['ref_prod']).sum()
#which RP with different ISIC
df_3457uniprodWHdiffISIC.loc[df_3457uniprodWHdiffISIC.duplicated(subset=['ref_prod']), :]

,ref_prod,activity,ISIC_cat,prod_freq
5,1-butanol,hydroformylation of propylene,2011:Manufacture of basic chemicals,3
10,1-propanol,1-propanol production,2011:Manufacture of basic chemicals,3
114,"acetone, liquid","phenol production, from cumene",2011:Manufacture of basic chemicals,11
374,"ammonia, anhydrous, liquid","synthetic fuel production, from coal, high tem...",1920:Manufacture of refined petroleum products,16
375,"ammonia, anhydrous, liquid","ammonia production, steam reforming, liquid",2012:Manufacture of fertilizers and nitrogen c...,16
...,...,...,...,...
12906,zinc,primary zinc production from concentrate,2420:Manufacture of basic precious and other n...,4
12908,zinc,"zinc scrap, post-consumer to generic market fo...","38:Waste collection, treatment and disposal ac...",4
12919,zinc concentrate,metalliferous hydroxide sludge to market for z...,3821:Treatment and disposal of non-hazardous w...,5
12925,zinc monosulfate,primary zinc production from concentrate,2420:Manufacture of basic precious and other n...,3


In [28]:
dupISIClist = df_3457uniprodWHdiffISIC.loc[df_3457uniprodWHdiffISIC.duplicated(subset=['ref_prod']), :].ref_prod.to_list()

In [29]:
df_3167uniprod.ref_prod.isin(dupISIClist)

0       False
1       False
2        True
3       False
4        True
        ...  
3162    False
3163     True
3164    False
3165    False
3166    False
Name: ref_prod, Length: 3167, dtype: bool

In [30]:
#now for those RP that has various ISIC, change the ISIC_cat for DF - df_3167uniprod
#https://kanoki.org/2019/07/17/pandas-how-to-replace-values-based-on-conditions/
df_3167uniprod.loc[df_3167uniprod.ref_prod.isin(dupISIClist),'ISIC_cat'] = 'various_ISIC'

In [31]:
df_3167uniprod

,ref_prod,activity,ISIC_cat,prod_freq
0,"1,1-difluoroethane, HFC-152a","1,1-difluoroethane production, HFC-152a",2011:Manufacture of basic chemicals,2
1,"1,1-dimethylcyclopentane",molecular sieve separation of naphtha,1920:Manufacture of refined petroleum products,2
2,1-butanol,"synthetic fuel production, from coal, high tem...",various_ISIC,3
3,1-pentanol,hydroformylation of butene,2011:Manufacture of basic chemicals,2
4,1-propanol,"synthetic fuel production, from coal, high tem...",various_ISIC,3
...,...,...,...,...
3162,zinc slag,"treatment of zinc slag, residual material land...",3822:Treatment and disposal of hazardous waste,1
3163,zinc sulfide,cobalt production,various_ISIC,3
3164,"zircon, 50% zirconium",heavy mineral sand quarry operation,0729:Mining of other non-ferrous metal ores,4
3165,zirconium oxide,zirconium oxide production,2011:Manufacture of basic chemicals,2


In [32]:
#which RP with new assigned "various ISIC"
df_3167uniprod.loc[df_3167uniprod['ISIC_cat']=='various_ISIC']

,ref_prod,activity,ISIC_cat,prod_freq
2,1-butanol,"synthetic fuel production, from coal, high tem...",various_ISIC,3
4,1-propanol,"synthetic fuel production, from coal, high tem...",various_ISIC,3
49,"acetone, liquid","synthetic fuel production, from coal, high tem...",various_ISIC,11
144,"ammonia, anhydrous, liquid",cobalt production,various_ISIC,16
152,ammonium sulfate,cobalt production,various_ISIC,6
...,...,...,...,...
3146,"yarn, jute","yarn production, jute",various_ISIC,3
3154,zinc,gold mine operation and refining,various_ISIC,4
3158,zinc concentrate,zinc mine operation,various_ISIC,5
3160,zinc monosulfate,zinc monosulfate production,various_ISIC,3


In [33]:
pd.DataFrame(df_3167uniprod.loc[df_3167uniprod['ISIC_cat']=='various_ISIC']).to_excel('197RP_WH_variousISIC.xlsx')

### extract only unique ref. product with >= 3 diff. activities

In [34]:
df_3167uniprod_sort = df_3167uniprod.sort_values(['ISIC_cat','prod_freq'])
df_3167uniprod_sort

,ref_prod,activity,ISIC_cat,prod_freq
216,"barley grain, feed, Swiss integrated production","barley grain, feed production, Swiss integrate...","0111:Growing of cereals (except rice), legumin...",1
842,"fava bean, Swiss integrated production","fava bean production, Swiss integrated product...","0111:Growing of cereals (except rice), legumin...",1
843,"fava bean, feed, Swiss integrated production","fava bean, feed production, Swiss integrated p...","0111:Growing of cereals (except rice), legumin...",1
1453,"maize grain, Swiss integrated production","maize grain production, Swiss integrated produ...","0111:Growing of cereals (except rice), legumin...",1
1455,"maize grain, feed, Swiss integrated production","maize grain, feed production, Swiss integrated...","0111:Growing of cereals (except rice), legumin...",1
...,...,...,...,...
1130,"heat, for reuse in municipal waste incineratio...","treatment of waste wire plastic, municipal inc...",various_ISIC,145
751,"electricity, medium voltage","electricity production, medium voltage, petrol...",various_ISIC,224
1129,"heat, district or industrial, other than natur...","particleboard production, uncoated, average gl...",various_ISIC,371
749,"electricity, low voltage","sawing and planing, paraná pine, kiln dried",various_ISIC,487


In [35]:
df_over3uniprod = df_3167uniprod_sort[(df_3167uniprod_sort.prod_freq >= 3)]
df_over3uniprod

,ref_prod,activity,ISIC_cat,prod_freq
1359,linseed,linseed production,"0111:Growing of cereals (except rice), legumin...",3
1454,"maize grain, feed","maize grain, feed production","0111:Growing of cereals (except rice), legumin...",3
1665,oat grain,oat production,"0111:Growing of cereals (except rice), legumin...",4
1786,peanut,peanut production,"0111:Growing of cereals (except rice), legumin...",4
2046,protein pea,protein pea production,"0111:Growing of cereals (except rice), legumin...",4
...,...,...,...,...
1130,"heat, for reuse in municipal waste incineratio...","treatment of waste wire plastic, municipal inc...",various_ISIC,145
751,"electricity, medium voltage","electricity production, medium voltage, petrol...",various_ISIC,224
1129,"heat, district or industrial, other than natur...","particleboard production, uncoated, average gl...",various_ISIC,371
749,"electricity, low voltage","sawing and planing, paraná pine, kiln dried",various_ISIC,487


In [36]:
pd.DataFrame(df_over3uniprod).to_excel('eidb_cutoff_748over3uniprod.xlsx')

In [37]:
df_over3uniprod.sort_values('prod_freq', ascending=False)[0:50]

,ref_prod,activity,ISIC_cat,prod_freq
743,"electricity, high voltage",cobalt production,various_ISIC,1895
749,"electricity, low voltage","sawing and planing, paraná pine, kiln dried",various_ISIC,487
1129,"heat, district or industrial, other than natur...","particleboard production, uncoated, average gl...",various_ISIC,371
751,"electricity, medium voltage","electricity production, medium voltage, petrol...",various_ISIC,224
1293,"land tenure, arable land, measured as carbon n...","clear-cutting, secondary forest to arable land...",68:Real estate activities,175
1290,"land tenure, arable land, measured as carbon n...","land already in use, perennial cropland to ann...",68:Real estate activities,150
1130,"heat, for reuse in municipal waste incineratio...","treatment of waste wire plastic, municipal inc...",various_ISIC,145
742,"electricity, for reuse in municipal waste inci...","treatment of digester sludge, municipal incine...",various_ISIC,141
1126,"heat, central or small-scale, other than natur...","heat production, anthracite, at stove 5-15kW",various_ISIC,132
1128,"heat, district or industrial, natural gas","heat and power co-generation, natural gas, con...",various_ISIC,116


In [38]:
# delete elec and heat production with over 100 diff. activities 
drop_elec_heat = [743, 749, 1129, 751, 1130, 742, 1126, 1128]

In [39]:
df_over3uniprod_wo_elec_heat = df_over3uniprod.drop(drop_elec_heat)
df_over3uniprod_wo_elec_heat

,ref_prod,activity,ISIC_cat,prod_freq
1359,linseed,linseed production,"0111:Growing of cereals (except rice), legumin...",3
1454,"maize grain, feed","maize grain, feed production","0111:Growing of cereals (except rice), legumin...",3
1665,oat grain,oat production,"0111:Growing of cereals (except rice), legumin...",4
1786,peanut,peanut production,"0111:Growing of cereals (except rice), legumin...",4
2046,protein pea,protein pea production,"0111:Growing of cereals (except rice), legumin...",4
...,...,...,...,...
2045,"protein feed, 100% crude","palm date production, conditioned and dried, o...",various_ISIC,46
1593,municipal solid waste,clinker production,various_ISIC,49
1233,"inorganic nitrogen fertiliser, as N",nutrient supply from diammonium phosphate,various_ISIC,54
1058,hard coal ash,clinker production,various_ISIC,61


In [40]:
#pd.DataFrame(df_over3uniprod_wo_elec_heat).to_excel('eidb_cutoff_over3uniprod_wo_elec_heat.xlsx')